In [1143]:
#pip installs
%pip install plotly
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1144]:
#libraries
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns

#plotting
import plotly.express as px
import plotly.offline as py

#linear regression
from sklearn.linear_model import LinearRegression
import datetime as dt

#prediction
from statsmodels.tsa.arima_model import ARIMA

In [1145]:
#load data from our csv file
#https://github.com/J535D165/CoronaWatchNL/tree/master/data-geo/data-national
cov_data = pd.read_csv('nl_cov_data/dutch_covid_data.csv')

#replace all empty values with 0's
cov_data = cov_data.fillna(0)

#remove row with column names
cov_data = cov_data.iloc[1: , :]

#get contaminations row only
cov_bes = cov_data.iloc[2::3, :]

#get deaths row only
cov_dea = cov_data.iloc[1::3, :]

#get hospital row only
cov_hos = cov_data.iloc[0::3, :]

#duplicates
cov_cum = cov_bes
dea_cum = cov_dea
hos_cum = cov_hos

#get date column
cov_bes_date = cov_bes.loc[:,"Datum"]
cov_dea_date = cov_dea.loc[:,"Datum"]
cov_hos_date = cov_hos.loc[:,"Datum"]

#convert to datetime
cov_bes_date = pd.to_datetime(cov_bes_date)
cov_dea_date = pd.to_datetime(cov_dea_date)
cov_hos_date = pd.to_datetime(cov_hos_date)

#get cumulative contaminations/other stats
cov_cum = cov_cum.loc[:,"AantalCumulatief"]
dea_cum = dea_cum.loc[:,"AantalCumulatief"]
hos_cum = hos_cum.loc[:,"AantalCumulatief"]

#get amount column
cov_bes = cov_bes.loc[:,"Aantal"]
cov_dea = cov_dea.loc[:,"Aantal"]
cov_hos = cov_hos.loc[:,"Aantal"]

#convert to numpy array
num_bes = np.array(cov_bes)
num_cum = np.array(cov_cum)
num_dea = np.array(cov_dea)
num_hos = np.array(cov_hos)
num_dea_cum = np.array(dea_cum)
num_hos_cum = np.array(hos_cum)

In [1146]:
# plot contaminations and cumulative contaminations
contaminations = px.bar(x=cov_bes_date, y=num_bes, height=550, 
               title='Contaminations per day')
contaminations.show()

contaminations_cum = px.bar(x=cov_bes_date, y=num_cum, height=550, 
               title='Cumulative contaminations in the Netherlands')
contaminations_cum.show()

In [1147]:
# plot deaths and cumulative deaths
deaths = px.bar(x=cov_dea_date, y=num_dea, height=550, 
               title='Deaths per day')
deaths.show()

deaths_cum = px.bar(x=cov_dea_date, y=num_dea_cum, height=550, 
               title='Cumulative deaths in the Netherlands')
deaths_cum.show()

In [1148]:
# plot deaths and cumulative deaths
hospital_num = px.bar(x=cov_hos_date, y=num_hos, height=550, 
               title='People entering the hospital per day')
hospital_num.show()

hospital_cum = px.bar(x=cov_hos_date, y=num_hos_cum, height=550, 
               title='Cumulative hospital intakes in the Netherlands')
hospital_cum.show()

When we want to say something about efficiency of measures and lockdowns, we first need to maintain a list of dates in which the covid regulations changed by big amounts.


12-03-2020: thuiswerkadvies, evenementen afgelast

13-03-2020: vliegverbod bepaalde landen

15-03-2020: sluiting horeca en scholen

23-03-2020: begin intelligente lockdown

11-05-2020: versoepelen contactberoepen

27-05-2020: diverse versoepelingen

30-06-2020: diverse versoepelingen

18-08-2020: iets meer maatregelen

27-09-2020: iets meer maatregelen

13-10-2020: lockdown, zware maatregelen

29-04-2020: versoepelingen


In [1149]:
#load data from our csv file
#from https://ourworldindata.org/grapher/covid-stringency-index
stri_data = pd.read_csv('nl_cov_data/covid-stringency-index.csv')

#filter out non dutch values
stri_data = stri_data[stri_data.Entity == "Netherlands"]

#get date data
stri_date = stri_data.loc[:,"Day"]


#convert to datetime
stri_date = pd.to_datetime(stri_date)

#get stringency index data
stri_index = stri_data.loc[:,"stringency_index"]

# plot stringency index
stringency = px.bar(x=stri_date, y=stri_index, height=550, 
               title='Graph of stringency index in the Netherlands')
stringency.show()

In [1150]:
groeifactoren = []

# calculating the growth factor
def groeifactor(delta_cn, delta_cn_old):
    """Returns the number of confirmed cases, Growth Factor=ΔC𝑛/ΔC𝑛−1"""
    return(delta_cn/delta_cn_old)

for i in range(len(cov_bes)):
    #we skip the first element, at is has no predecessor
    if i != 0:
        groeifact = groeifactor(num_bes[i], num_bes[i-1])
        groeifactoren.append(groeifact)

#convert list to numpy array
groeifactoren = np.array(groeifactoren)

#remove first element of dates as this has no r number
cov_bes_date_g = cov_bes_date[1:]

g_grafiek = px.bar(x=cov_bes_date_g, y=groeifactoren, height=550, 
               title='Evolutie groeifactor covid-19')
g_grafiek.show()

In [1151]:
#there is one less data point in our dataset for the first day of contaminations, we adjust this difference for synchronous data
num_bes = num_bes[7:]
num_dea = num_dea[8:]
num_hos = num_hos[8:]
cov_bes_date_g = cov_bes_date_g[6:]

In [1152]:
#remove the 25th of march as this data was not collected properly by the RIVM due to technical problems
outlier = int(cov_bes_date_g.index[cov_bes_date_g == '2021-03-25'].tolist()[0] / 3)
num_bes = np.delete(num_bes, 383)
num_dea = np.delete(num_dea, 383)
num_hos = np.delete(num_hos, 383)
cov_bes_date_g = cov_bes_date_g.loc[cov_bes_date_g != '2021-03-25']

# #DEBUG: check if data is synched
# print(len(num_bes))
# print(len(num_dea))
# print(len(num_hos))
# print(len(cov_bes_date_g))

In [1153]:
#update the stringency index list to fit our time period and remove the 24th and 25th of march
#timeframe = 2020-03-12 until 2020-04-17

#get boundaries and exemptions indexes from dataset
begin = stri_date.index[stri_date == '2020-03-06'].tolist()[0] #81944
end = stri_date.index[stri_date == '2021-04-17'].tolist()[0] #82351
ma24 = stri_date.index[stri_date == '2021-03-24'].tolist()[0] #82327

#adjust timeframe of stringency index dates to contaminations timeframe
stri_date = stri_date.loc[stri_date >= '2020-03-06']
stri_date = stri_date.loc[stri_date <= '2021-04-17']

#adjust timeframe of stringency index dates to remove wrong data points in other dataset
stri_date = stri_date.loc[stri_date != '2021-03-24']
stri_date = stri_date.loc[stri_date != '2021-03-25']

#adjust the stringency index data to above frame
stri_index = stri_index.iloc[45:]
stri_index = stri_index.iloc[:408]

#remove missing elements from dataframe in context
stri_index = stri_index.drop(stri_index.index[384])
stri_index = stri_index.drop(stri_index.index[384])


In [1154]:
#aquire and adjust data about the R number
# The data about the groeifactor was gotten from https://data.rivm.nl/covid-19/COVID-19_reproductiegetal.json
# note that this data is in .json format, we have converted it using http://convertcsv.com/json-to-csv.htm
g_data = pd.read_csv('nl_cov_data/reproductiegetallen.csv')

#get date data
g_date = g_data.loc[:,"Date"]

#convert to datetime
g_date = pd.to_datetime(g_date)

#get average R data
g_data = g_data.loc[:,"Rt_avg"]

#plot data
g_grafiek = px.bar(x=g_date, y=g_data, height=550, 
               title='Evolutie R getal covid-19')
g_grafiek.show()

In [1155]:
#update the reprodutctiegetallen list to fit our time period and remove the 24th and 25th of march
#timeframe = 2020-03-12 until 2020-04-17

#get boundaries and exemptions indexes from dataset
begin = g_date.index[g_date == '2020-03-06'].tolist()[0] #18
end = g_date.index[g_date == '2021-04-17'].tolist()[0] #425
ma24 = g_date.index[g_date == '2021-03-24'].tolist()[0] #401 

#adjust timeframe of reproductiegetallen dates to contaminations timeframe
g_date = g_date.loc[g_date >= '2020-03-06']
g_date = g_date.loc[g_date <= '2021-04-17']

#adjust timeframe of reproductiegetallen dates to remove wrong data points in other dataset
g_date = g_date.loc[g_date != '2021-03-24']
g_date = g_date.loc[g_date != '2021-03-25']

#adjust the reproductiegetallen data to above frame
g_data = g_data.iloc[18:]
g_data = g_data.iloc[:408]

#remove missing elements from dataframe in context
g_data = g_data.drop(g_data.index[401])
g_data = g_data.drop(g_data.index[401])

In [1156]:
#DEBUG
#assure all data is of equal size
print(len(num_bes))
print(len(num_dea))
print(len(num_hos))
print(len(cov_bes_date_g))
print(len(g_data))
print(len(g_date))
print(len(stri_index))
print(len(stri_date))

#DATA explanation
# num_bes = amount of contaminations due to covid-19 in the Netherlands
# num_dea = amount of deaths due to covid-19 in the Netherlands
# num_hos = amount of people taken into the hospital due to covid-19 in the Netherlands
# stri_index = stringency of covid-19 measures in the Netherlands
# g_data = R measure of infectivity covid-19 in the Netherlands
# cov_bes_date_g/g_date/stri_date = timeframe used for our work

406
406
406
406
406
406
406
406


In [1157]:
#Before making a model it is very neccessary to take some weekly averages for our model to use for decision making
#We get average over the last 7 days for all our data points, as it takes approximatly this long for covid-19 measures to take effect
avg_r = g_data.rolling(7).mean()
avg_str = stri_index.rolling(7).mean()

#to get an average of our numpy arrays we need to convert them to pandas dataframes (from np.array)
avg_bes = pd.DataFrame(num_bes).rolling(7).mean()
avg_dea = pd.DataFrame(num_dea).rolling(7).mean()
avg_hos = pd.DataFrame(num_hos).rolling(7).mean()

#to use 7 day means, we need to trim of 6 results, as the first 6 results have no 7 day mean
avg_r = avg_r.iloc[6:]
avg_str = avg_str.iloc[6:]
avg_bes = avg_bes.iloc[6:]
avg_dea = avg_dea.iloc[6:]
avg_hos = avg_hos.iloc[6:]

#because we use 7 day means, we also need to trim the normal data
g_data = g_data.iloc[6:]
stri_index = stri_index.iloc[6:]
num_bes = num_bes[6:]
num_dea = num_dea[6:]
num_hos = num_hos[6:]

#we also have to trim our 3 date objects
cov_bes_date_g = cov_bes_date_g.iloc[6:]
g_date = g_date.iloc[6:]
stri_date = stri_date.iloc[6:]

In [1158]:
#Before making a model, we find it very handy to take delta's from our data, so our model can use delta's as input
#This because in our humble opinion, delta's are a better grade of data adjustment, as they show how rapidly things go wrong (or good)
del_r = g_data.diff().shift(-1)
del_str = stri_index.diff().shift(-1)

#to get an delta of our numpy arrays we need to convert them to pandas dataframes (from np.array)
del_bes = pd.DataFrame(num_bes).diff().shift(-1)
del_dea = pd.DataFrame(num_dea).diff().shift(-1)
del_hos = pd.DataFrame(num_hos).diff().shift(-1)

#because of deltas we need to remove the last element of everything
del_bes = del_bes.iloc[:len(del_bes)-1]
del_dea = del_dea.iloc[:len(del_dea)-1]
del_hos = del_hos.iloc[:len(del_hos)-1]
del_r = del_r.iloc[:len(del_r)-1]
del_str = del_str.iloc[:len(del_str)-1]

#we also compensate all other elements
lengthmin = len(avg_r) - 1
avg_str = avg_str.iloc[:lengthmin]
avg_bes = avg_bes.iloc[:lengthmin]
avg_dea = avg_dea.iloc[:lengthmin]
avg_hos = avg_hos.iloc[:lengthmin]
g_data = g_data.iloc[:lengthmin]
stri_index = stri_index.iloc[:lengthmin]
num_bes = num_bes[:lengthmin]
num_dea = num_dea[:lengthmin]
num_hos = num_hos[:lengthmin]
cov_bes_date_g = cov_bes_date_g.iloc[:lengthmin]
g_date = g_date.iloc[:lengthmin]
stri_date = stri_date.iloc[:lengthmin]
avg_r = avg_r.iloc[:lengthmin]

In [1174]:
# We do all these index resets because we want to be able to freely merge panda's series and dataframes
# This because we want our eventual possible in and outputs in one single dataframe.

# Reset the series indexes of all averages
avg_bes.reset_index(drop=True, inplace=True)
avg_str.reset_index(drop=True, inplace=True)
avg_dea.reset_index(drop=True, inplace=True)
avg_hos.reset_index(drop=True, inplace=True)
avg_r.reset_index(drop=True, inplace=True)

# Reset the series indexes of all data
# Note that the num_ data is not reset, as it is a numpy, not a panda dataframe object
stri_index.reset_index(drop=True, inplace=True)
g_data.reset_index(drop=True, inplace=True)

# Reset the series indexes of all deltas
del_bes.reset_index(drop=True, inplace=True)
del_str.reset_index(drop=True, inplace=True)
del_dea.reset_index(drop=True, inplace=True)
del_hos.reset_index(drop=True, inplace=True)
del_r.reset_index(drop=True, inplace=True)

# Reset the series indexes of all dates
cov_bes_date_g.reset_index(drop=True, inplace=True)
g_date.reset_index(drop=True, inplace=True)
stri_date.reset_index(drop=True, inplace=True)


In [1176]:
np_bes = avg_bes.to_numpy()
np_bes = np.concatenate(np_bes, axis=0)
mean_graph = px.bar(x=cov_bes_date_g, y=np_bes, height=550, 
               title='Evolutie gemiddelde besmettingen in 7 dagen')
mean_graph.show()

np_dbes = del_bes.to_numpy()
np_dbes = np.concatenate(np_dbes, axis=0)
mean_graph = px.bar(x=cov_bes_date_g, y=np_dbes, height=550, 
               title='Evolutie delta besmettingen in 7 dagen')
mean_graph.show()

In [1183]:
#creating one panda dataframe for all covid inputs
cov_data = pd.concat([pd.DataFrame(num_bes), pd.DataFrame(num_dea), pd.DataFrame(num_hos)], axis=1, ignore_index=True)
all_data = pd.concat([cov_data, avg_bes, avg_dea, avg_hos, del_bes, del_dea, del_hos, g_data, avg_r, del_r, stri_index],
                        axis=1, ignore_index=True)

In [1188]:
# We shuffle the dataframe
all_data_shuffled = all_data.sample(frac=1)

# We reset the indexes that were present before shuffle
all_data_shuffled = all_data_shuffled.reset_index(drop=True)

#! DISABLED PART DUE TO OVERWRITING CSV
# Because of the random shuffle, it is impossible to reproduce our results
# Hence we shuffled only once and afterwards saved the shuffled file as csv
# This csv is loaded in when doing our calculations and therefore our results are reproducible

# ! UNCOMMENT THIS TO PRINT CSV DATA
# print(all_data_shuffled.to_csv(index=False))

0,1,2,3,4,5,6,7,8,9,10,11,12
3011.0,13.0,46.0,2758.714285714286,14.571428571428571,34.714285714285715,283.0,0.0,2.0,1.26,1.2899999999999998,-0.020000000000000018,62.04
7633.0,29.0,95.0,7075.571428571428,27.142857142857142,73.28571428571429,194.0,-8.0,-29.0,0.97,0.9714285714285714,-0.010000000000000009,75.0
4394.0,72.0,72.0,3410.285714285714,61.142857142857146,65.0,-53.0,-7.0,11.0,1.1,1.0357142857142858,0.039999999999999813,78.7
6521.0,39.0,60.0,5302.857142857143,48.0,61.285714285714285,258.0,-14.0,-8.0,1.24,1.23,-0.040000000000000036,56.48
1844.0,4.0,20.0,1668.4285714285713,3.5714285714285716,18.0,373.0,-2.0,6.0,1.16,1.1714285714285713,-0.010000000000000009,48.15
108.0,21.0,34.0,180.71428571428572,29.285714285714285,33.57142857142857,90.0,12.0,-20.0,1.08,0.9771428571428571,-0.1200000000000001,71.3
925.0,2.0,5.0,663.8571428571429,2.7142857142857144,6.285714285714286,-128.0,-2.0,4.0,1.28,1.342857142857143,-0.050000000000000044,50.93
6396.0,22.0,58.0,5095.571428571428,34.714285714285715,7

In [ ]:
#linear regression model
# we want to predict the stringency index
# we want to predict this using contaminations, hospital admissions, deaths and the R number
# we have also calculated means and delta's to use as inputs for all of the above inputs

#we want to make a pandas dataframe X of the input data
X = cov_data
#we want to make a pandas dataframe Y of the output data
Y = stri_index

#we use the LinearRegression() method to create a linear regression object
#TODO: #14 split training and test, pick evaluation method
#TODO: #15 random sampling into one dataframe, save to .csv file as random otherwises makes results not reproducible
lin_mod = LinearRegression()
lin_mod.fit(X, Y)

#do a prediction
predicted_stri = lin_mod.predict([[5500, 140, 500]])
print(predicted_stri)
#TODO: predict stringency index based on inputs: besmettingen, ziekenhuisopnames, doden en R getal
#TODO: #2 algoritme 1: linear regression
#TODO: #3 algoritme 2: C4.5
#TODO: #4 optimaliseer beide algoritmes
#TODO: #5 vergelijk en schrijf paper van 6-8 kantjes

[89.40602317]
